In [1]:
import h5py
import numpy as np

In [10]:
def angular_band(pos, angle, width, xx, zz):
    return (pos-xx < angle*zz) * (pos + width - xx > angle*zz)

def angular_bands_on_drosophila(pos, width, xx,zz):
    angle = (pos/100.0 - 0.5)/3.
    return angular_band(pos, angle, width, xx, zz)

In [21]:
with h5py.File("runs/tenth_egg.hdf5", "r") as f:
    attrs = dict(f.attrs)
    poss = np.array(f["x"])
    cell_properties = np.array(f["properties"][0])
    p = np.array(f["p"])
    q = np.array(f["q"])




xx = poss[-1][:,0]
yy = poss[-1][:,1]
zz = poss[-1][:,2]

print(xx.min(), xx.max())
print(yy.min(), yy.max())
print(zz.min(), zz.max())

xx = (xx - xx.min())/(xx.max() - xx.min())*100
yy = (yy - yy.min())/(yy.max() - yy.min())*100
zz = (zz - zz.min())/(zz.max() - zz.min())*100

AB = lambda pos, width: angular_bands_on_drosophila(pos, width, xx, zz)

prop2 = np.zeros(cell_properties.shape)


width = 4
dist = 30

midline = 0
gb_mask = (xx > 30)*(zz < 67)*(((yy > 50+midline) + (yy < 50-midline) )>0)

prop2[gb_mask] = 1.0 # Germ Band


for i in range(1,3):   #7
    xlow = 8 + i*dist
    w = width*(1 + 1)

    # prop2[(xx > xlow)*(xx < xhigh)*(zz > 30)] = 5.0 # 

    # prop2[AB(xlow, w)*(zz>57)] = 5.0 # Dorsal Folds
    prop2[AB(xlow, w)*(zz>65)] = 5.0 # Dorsal Folds



prop2[AB(15, 5)*(zz<97)*(zz>45)] = 5.0 # Cephalic Furrow

# prop2[(xx > 30)*(xx < 97)*(zz < 50)*(yy< 50 + 40) * (yy > 50- 40)] = 1.0 # red
# prop2[(xx > 85)*(xx < 92)] = 4.0 # light blue tip

# prop2[angular_band(108, 0.3, 15, xx,zz)] = 3.0 # (PMG) Posterior Midgut Invagination
prop2[(yy < 65)*(yy > 35)*(xx > 15)*(xx < 110)*(zz < 50)] = 4.0 # Ventral Furrow
prop2[(yy < 65)*(yy > 35)*(xx > 15)*(xx < 90)*(zz < 50)] = 2.0 # Ventral Furrow

prop2[angular_band(106, 0.2, 12, xx,zz)] = 4.0 # (PMG) Posterior Midgut Invagination

# 
# prop2[angular_band(112, 0.3, 10, xx,zz)] = 4.0 # (PMG) Posterior Midgut Invagination
# prop2[(xx>97)*(zz>68)*(yy < 53)*(yy>47)] = 3.0 # (PMG) Posterior Midgut Invagination

# prop2[(xx > 94)*(zz>63)*(yy<50+16)*(yy>50-16)] = 3.0 # yellow tip 


properties = prop2


with h5py.File("runs/daniel_alber_iso_notip.hdf5", "w") as f:
    create = lambda name, data: f.create_dataset(name, data=data)

    create("x", poss)

    create("properties", [properties])

    create("p", p)

    create("q", q)

    f.attrs.update(attrs)


-79.91615 59.94943
-26.649378 26.66619
-26.649714 26.656801


In [3]:
with h5py.File("runs/pressure_run.hdf5", 'r') as f:
    positions = f['x'][::1]
    loaded_properties = None
    if 'properties' in f:
        loaded_properties = f['properties'][:]

    if "tosave" in f:
        tosave = f['tosave'][:]
        print(tosave.shape)
        print(positions.shape)
    else:
        tosave = None

    ps = f['p'][::1]
    qs = f['q'][::1]

(901, 5000)
(901, 5000, 3)


In [ ]:
# make 10_000 random points in a sphere

n = 10_000

r = np.random.uniform(0, 30, n)

phi, theta = np.random.uniform(0, 2*np.pi, n), np.arccos(2*np.random.uniform(0, 1, n) - 1)

x = r*np.sin(theta)*np.cos(phi)
y = r*np.sin(theta)*np.sin(phi)
z = r*np.cos(theta)

poss = np.array([x, y, z]).T

properties = np.zeros(n)

## p and q are n random unit vectors
p = np.random.normal(size=(n, 3)) 
p /= np.linalg.norm(p, axis=1)[:, None]

q = np.random.normal(size=(n, 3))
q /= np.linalg.norm(q, axis=1)[:, None]

with h5py.File("runs/random_sphere.hdf5", "w") as f:
    create = lambda name, data: f.create_dataset(name, data=data)

    create("x", poss)

    create("properties", [properties])

    create("p", p)

    create("q", q)


In [37]:
def make_disc(N, radius=10):
    x = np.random.randn(N, 3)
    x[:,-1] = 0.0
    r = radius * np.random.rand(N)**(1/2.)
    x /= np.sqrt(np.sum(x**2, axis=1))[:, None]
    x *= r[:, None]

    p = np.repeat(np.array([[0.0,0.0,1.0]]),N, axis=0)
    # q = np.repeat(np.array([[1.0,0.0,0.0]]),N, axis=0)
    q = np.random.random((N, 3))
    q /= np.linalg.norm(q, axis=1)[:, None]

    mask = np.ones(N)

    disc_data = (mask, x, p, q)

    mask[(x[:,0] < 10)*(x[:,0] > -10)] = 0
    return disc_data


N = 5000

properties, x, p, q = make_disc(N, 100)


with h5py.File("runs/disc.hdf5", "w") as f:
    create = lambda name, data: f.create_dataset(name, data=data)

    create("x", [x])

    create("properties", [properties])

    create("p", p)

    create("q", q)



In [10]:
with h5py.File("runs/random_baby_3.hdf5", "r") as f:
    positions = f["x"][::2,:,:]

steps, N, _ = positions.shape
print(positions.shape)
# # flatten the positions
positions = positions.reshape(steps*N, 3)

# add the number og steps and N to the beginning of the array
properties = np.zeros((1, 3), dtype=float)

properties[0, 0] = N
properties[0, 1] = steps
properties[0, 2] = 0


savedata = np.concatenate([properties, positions], axis=0)


print(savedata.shape)
# # save the positions to a csv file
np.savetxt("runs/random_baby.csv", savedata, delimiter=",")

(101, 10000, 3)
(1010001, 3)
